# Decorator
- Decorators are the functions that take another function as an argument, add some kinds of functionality and return the function without altering the function that was passed in
- In the ```wrapper``` function, it must execute the ```original``` function and return what ```original``` function returns

In [7]:
def decorator_function(original_func):
    def wrapper_func():
        return original_func()
    return wrapper_func

def display():
    print('display() func ran')

dec = decorator_function(display)
dec()

display() func ran


In [11]:
print(dec.__closure__)
print(dec.__closure__[0].cell_contents)

(<cell at 0x000001A1C257A970: function object at 0x000001A1C26560D0>,)
<function display at 0x000001A1C26560D0>


In [12]:
dec.__closure__[0].cell_contents()

display() func ran


In [13]:
import dis
dis.dis(dec)

  3           0 LOAD_DEREF               0 (original_func)
              2 CALL_FUNCTION            0
              4 RETURN_VALUE


- The syntax ```@decorator_function_name``` is added on the line before the function header will applied the decorator function

In [14]:
@decorator_function
def display():
    print('display() func ran')

display()

display() func ran


- Adding some functionality to alter the ```original``` function inside the ```wrapper_function```

In [36]:
def decorator_function(original_func):
    def wrapper_func():
        print(f'wrapper function executes this before {original_func.__name__}')
        return original_func()
    return wrapper_func

@decorator_function
def display():
    print('display() func ran')
    
display()

wrapper function executes this before display
display() func ran


In [20]:
@decorator_function
def display_info(name, age):
    print(f'display() function ran with arguments name={name}, age={age}')

display_info('tuan', 30)

TypeError: wrapper_func() takes 0 positional arguments but 2 were given

- ```wrapper_function()``` takes 0 positional arguments but 2 were given because 2 arguments ```(name, age)``` were passed to the ```display_info()``` function while the definition of ```wrapper_function()``` takes 0 argument
- Adding ```*args, **kwargs to wrapper_function()``` definition to accept any positional arguments and keyword arguments

In [31]:
def decorator_function(original_func):
    def wrapper_func(*args, **kwargs):
        print(f'wrapper function executes this before {original_func.__name__}')
        return original_func(*args, **kwargs)
    return wrapper_func

@decorator_function
def display_info(name, age):
    '''Display info, takes argument: name & age
    Return: None
    '''
    print(f'display_info() function ran with arguments name={name}, age={age}')

display_info('tuan', 30)

wrapper function executes this before display_info
display_info() function ran with arguments name=tuan, age=30


In [26]:
display_info

<function __main__.decorator_function.<locals>.wrapper_func(*args, **kwargs)>

In [27]:
display_info.__closure__[0].cell_contents

<function __main__.display_info(name, age)>

In [28]:
display_info.__closure__[0].cell_contents('tuan', 30)

display_info() function ran with arguments name=tuan, age=30


In [29]:
dis.dis(display_info)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_CONST               1 ('wrapper function executes this before ')
              4 LOAD_DEREF               0 (original_func)
              6 LOAD_ATTR                1 (__name__)
              8 FORMAT_VALUE             0
             10 BUILD_STRING             2
             12 CALL_FUNCTION            1
             14 POP_TOP

  4          16 LOAD_DEREF               0 (original_func)
             18 LOAD_FAST                0 (args)
             20 BUILD_MAP                0
             22 LOAD_FAST                1 (kwargs)
             24 DICT_MERGE               1
             26 CALL_FUNCTION_EX         1
             28 RETURN_VALUE


In [30]:
dis.dis(decorator_function)

  2           0 LOAD_CLOSURE             0 (original_func)
              2 BUILD_TUPLE              1
              4 LOAD_CONST               1 (<code object wrapper_func at 0x000001A1C26623A0, file "C:\Users\nmtuan\AppData\Local\Temp/ipykernel_28120/3752065882.py", line 2>)
              6 LOAD_CONST               2 ('decorator_function.<locals>.wrapper_func')
              8 MAKE_FUNCTION            8 (closure)
             10 STORE_FAST               1 (wrapper_func)

  5          12 LOAD_FAST                1 (wrapper_func)
             14 RETURN_VALUE

Disassembly of <code object wrapper_func at 0x000001A1C26623A0, file "C:\Users\nmtuan\AppData\Local\Temp/ipykernel_28120/3752065882.py", line 2>:
  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_CONST               1 ('wrapper function executes this before ')
              4 LOAD_DEREF               0 (original_func)
              6 LOAD_ATTR                1 (__name__)
              8 FORMAT_VALUE           

In [34]:
print(display_info.__doc__)
print(display_info.__name__)

None
wrapper_func


- Decorator doesn’t keep the ```original``` function metadata
- Python provides functools module to support for keeping the ```original``` function metadata
- Adding ```@wraps(original_function) before any wrapper``` functions in the decorator to keep ```original``` function metadata

In [37]:
from functools import wraps

def decorator_function(original_func):
    
    @wraps(original_func)
    def wrapper_func(*args, **kwargs):
        print(f'wrapper function executes this before {original_func.__name__}')
        return original_func(*args, **kwargs)
    return wrapper_func

@decorator_function
def display():
    '''Display take no argument. Returns None'''
    print('display() func ran')

@decorator_function
def display_info(name, age):
    '''Display info, takes argument: name & age
    Return: None
    '''
    print(f'display_info() function ran with arguments name={name}, age={age}')

display_info('tuan', 30)

wrapper function executes this before display_info
display_info() function ran with arguments name=tuan, age=30


In [38]:
display_info.__name__

'display_info'

In [40]:
print(display_info.__doc__)

Display info, takes argument: name & age
    Return: None
    


## Class decorator
- Class decorator will play the same role as function decorator
- The ```__init__()``` method will map the ```original``` function to the instance of the class decorator
- The ```__call__()``` method will act the same as ```wrapper``` function()

In [57]:
class decorator_class():
    def __init__(self, orig_func):
        self.original_function = orig_func
    
    def __call__(self, *args, **kwargs):
        print(f'call() method executes this before {self.original_function.__name__}')
        return self.original_function(*args, **kwargs)
    
@decorator_class
def display():
    '''Display take no argument. Returns None'''
    print('display() func ran')

@decorator_class
def display_info(name, age):
    '''Display info, takes argument: name & age
    Return: None
    '''
    print(f'display_info() function ran with arguments name={name}, age={age}')

display_info('tuan', 30)

call() method executes this before display_info
display_info() function ran with arguments name=tuan, age=30


In [58]:
print(display_info.__doc__)
print(display_info.__name__)

None


AttributeError: 'decorator_class' object has no attribute '__name__'

In [52]:
from functools import wraps, update_wrapper

class decorator_class():
    def __init__(self, orig_func):
        self.original_function = orig_func
        update_wrapper(self, orig_func)
    
    def __call__(self, *args, **kwargs):
        print(f'call() method executes this before {self.original_function.__name__}')
        return self.original_function(*args, **kwargs)
    
@decorator_class
def display():
    '''Display take no argument. Returns None'''
    print('display() func ran')

@decorator_class
def display_info(name, age):
    '''Display info, takes argument: name & age
    Return: None
    '''
    print(f'display_info() function ran with arguments name={name}, age={age}')

display_info('tuan', 30)

call() method executes this before display_info
display_info() function ran with arguments name=tuan, age=30


In [56]:
print(display_info.__doc__)
print(display_info.__name__)

Display info, takes argument: name & age
    Return: None
    
display_info
